In [2]:
%cd Self-Correction-Human-Parsing

C:\Users\multicampus\Desktop\자율PJT\S07P31B302\AI\Python\Self-Correction-Human-Parsing


In [ ]:
!pip install opencv-python

In [3]:
dataset = 'lip'

In [6]:
import gdown

In [7]:
if dataset == 'lip':
    url = 'https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH'
elif dataset == 'atr':
    url = 'https://drive.google.com/uc?id=1ruJg4lqR_jgQPj-9K0PP-L2vJERYOxLP'
elif dataset == 'pascal':
    url = 'https://drive.google.com/uc?id=1E5YwNKW2VOEayK9mWCS3Kpsxf-3z04ZE'

output = 'checkpoints/final.pth'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH
To: C:\Users\multicampus\Desktop\자율PJT\S07P31B302\AI\Python\Self-Correction-Human-Parsing\checkpoints\final.pth
100%|██████████| 267M/267M [00:44<00:00, 5.99MB/s] 


'checkpoints/final.pth'

In [15]:
!python simple_extractor.py --dataset 'lip' --model-restore 'checkpoints/final.pth' --input-dir 'inputs' --output-dir 'outputs'

C:\ProgramData\Anaconda3\envs\Python\lib\site-packages\torch\utils\cpp_extension.py:287: UserWarning: Error checking compiler version for cl: [WinError 2] 지정된 파일을 찾을 수 없습니다
  warnings.warn('Error checking compiler version for {}: {}'.format(compiler, error))
Traceback (most recent call last):
  File "simple_extractor.py", line 24, in <module>
    import networks
  File "C:\Users\multicampus\Desktop\자율PJT\S07P31B302\AI\Python\Self-Correction-Human-Parsing\networks\__init__.py", line 3, in <module>
    from networks.AugmentCE2P import resnet101
  File "C:\Users\multicampus\Desktop\자율PJT\S07P31B302\AI\Python\Self-Correction-Human-Parsing\networks\AugmentCE2P.py", line 21, in <module>
    from modules import InPlaceABNSync
  File "C:\Users\multicampus\Desktop\자율PJT\S07P31B302\AI\Python\Self-Correction-Human-Parsing\modules\__init__.py", line 1, in <module>
    from .bn import ABN, InPlaceABN, InPlaceABNSync
  File "C:\Users\multicampus\Desktop\자율PJT\S07P31B302\AI\Python\Self-Correction-Hum